In [1]:
print("OK")

OK


In [2]:
%pwd

'/Users/nihal/Downloads/End-End-MediRAG-GEN-AI/research'

In [1]:
import os
import langchain_core
print(dir(langchain_core))

['VERSION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_api', '_import_utils', 'surface_langchain_beta_warnings', 'surface_langchain_deprecation_warnings', 'version']


In [4]:
os.chdir('/Users/nihal/Downloads/End-End-MediRAG-GEN-AI')

In [2]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  

/Users/nihal/anaconda3/envs/llmapp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)  # it extracts info using this and using glob="*.pdf" it only loads pdf files and later it gives docs

    documents = loader.load()

    return documents

In [4]:
# Check if Data directory exists, if not, use the correct path
import os
data_path = 'Data/' if os.path.exists('Data/') else '/Users/nihal/Downloads/End-End-MediRAG-GEN-AI/Data/'
extracted_data = load_pdf_file(data=data_path)


In [5]:
#extracted_data

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 5860


In [8]:
#text_chunks

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [10]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

/var/folders/g5/54vbnb5j1gdc46kk4fr0tvmr0000gn/T/ipykernel_29048/4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


Length 384


In [13]:
#query_result


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medicalbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [16]:
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [17]:
from langchain_pinecone import PineconeVectorStore
# we embed each chunk and upsert the embeddings into our Pinecone Index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings, 
    index_name=index_name
)

In [18]:
#loading existing index

from langchain_pinecone import PineconeVectorStore
# we embed each chunk and upsert the embeddings into our Pinecone Index
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings, 
    index_name=index_name
)

In [19]:
docsearch

In [20]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [21]:
retrieved_docs = docsearch.similarity_search("What is diabetes?")

In [22]:
retrieved_docs

[Document(id='116955f8-e361-4dba-b050-6a208ac55ab6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 542.0, 'page_label': '543', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='begin to fall. A person with diabetes mellitus either does\nnot make enough insulin, or makes insulin that does not\nwork properly. The result is blood sugar that remains\nhigh, a condition called hyperglycemia.\nDiabetes must be diagnosed as early as possible. If\nleft untreated, it can damage or cause failure of the eyes,\nkidneys, nerves, heart, blood vessels, and other body\norgans. Hypoglycemia, or low blood sugar, may also be\ndiscovered through blood sugar testing. Hypoglycemia is'),
 Document(id='f3844b42-7633-432d-a6f0-dd3d53639b29', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 542.0, 'page

In [31]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0.4, max_tokens=500)



In [24]:
import langchain
print(langchain.__version__)

1.2.4


In [32]:
from langchain_classic.chains import LLMChain


In [33]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assitant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [37]:
response = rag_chain.invoke({"input": "What is diabetes?"})
print(response["answer"])

Diabetes is a chronic condition where the body either does not produce enough insulin or the insulin produced does not work effectively, leading to high blood sugar levels, a condition known as hyperglycemia. If left untreated, diabetes can cause serious damage to various organs, including the eyes, kidneys, nerves, heart, and blood vessels. Early diagnosis and management are crucial to prevent complications.
